In [1]:
import cv2
import time
import numpy as np

first_frame = None
text = 'Nothing to see...'


def get_video_input(movie_file=None):
    if movie_file is None:
        cap = cv2.VideoCapture(0)
        time.sleep(0.25)
 
    # otherwise, we are reading from a video file
    else:
        cap = cv2.VideoCapture(movie_file)  

    w, h = (640, 400)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, w)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, h)

    return cap


def process_frame(frame):
    global first_frame

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    return frame #cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

def run(movie_file, save=False):
    global text

    is_fst_frame = True
    cap = get_video_input(movie_file)
    frames = []
    
    # main loop
    while True:
        ret, frame = cap.read()
        if is_fst_frame:
            is_fst_frame = False
            continue
        # print(frame.shape)
        # if the frame could not be grabbed, then we have reached the end
        # of the video
        if not ret:
            break
        
        text = 'Nothing to see...'
 
        frame_delta = process_frame(frame)

        # draw the text on the frame
        #cv2.putText(frame, "Status: {}".format(text), (10, 20),
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # show the frame and record if the user presses a key
        cv2.imshow("Video Stream", frame)
        cv2.imshow("Frame Delta", frame_delta)

        frames.append((frame, frame_delta))
        if cv2.waitKey(5) == 27:
            break

    if save: 
        for idx, f_triplet in enumerate(frames):
            f, t, fd = f_triplet
            if idx % 4 == 0:
                cv2.imwrite('./data/frame_{num:03d}.png'.format(num=idx), f)
                cv2.imwrite('./data/framed_{num:03d}.png'.format(num=idx), fd)
        # create the animated gif with:
        # convert -loop 0 -scale 75% $(ls ./data/frame_*.png | sort) ./data/movement.gif

    # cap.release()
    cv2.destroyAllWindows()

In [ ]:
run(0)